<a href="https://colab.research.google.com/github/VioletRaven/COVID-19-Spark-Big-Data-Analysis/blob/main/QUERY_2_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [ ]:
#funziona ma non su colab
#import locale
#export LC_ALL="it_IT.UTF-8"
#export LC_CTYPE="it_IT.UTF-8"
#sudo dpkg-reconfigure locales

#locale.setlocale(locale.LC_ALL, 'it_IT')

In [ ]:
import findspark

findspark.init()
findspark.find()

from pyspark import SparkContext
from datetime import datetime
from calendar import monthrange

In [ ]:
# Inizializzo lo Spark Context
sc = SparkContext("local[2]", "query_2")

In [ ]:
#Q2 
#Utilizzando somministrazioni-vaccini-latest.csv, per le donne, per ogni fascia anagrafica, per ogni mese solare e per ogni 
#area (o regione), determinare il minimo, il massimo e la media aritmetica del numero di vaccinazioni effettuate nel mese in esame.
#Per la risoluzione della query, considerare le sole fasce anagrafiche per cui nel mese solare in esa- me vengono registrati almeno 
#due giorni di campagna vaccinale. Considerare i dati a partire dall’1 Gennaio 2021.
#Esempio di output:
#Gennaio, 20-29, Lazio, 20, 50, 42.8 (assumendo che 20 sia il minimo, 50 il massimo e 42.8 il numero medio di vaccinazioni effettuate nel mese di Gennaio dalla regione Lazio per la fascia d’eta 20-29)
#Gennaio, 30-39, Lazio, 30, 60, 51.7 (assumendo che 30 sia il minimo, 60 il massimo e 51.7 il numero medio di vaccinazioni effettuate nel mese di Gennaio dalla regione Lazio per la fascia d’eta` 30-39) ...

In [ ]:
# Carico il dataset n°1 somministrazioni-vaccini-latest.csv
# Creo un RDD
data = sc.textFile("/content/drive/MyDrive/Colab Notebooks/somministrazioni-vaccini-latest.csv")

#Identifico la prima riga che contiene le intestazioni e la escludo dall'RDD
header = data.first()
data = data.filter(lambda x: x != header)

In [ ]:
#Step 0 Splitto l'RDD del dataset n°1 per il separatore di campo virgola
#Step 1 Filtro l'RDD ottenuto per avere solo i dati compresi tra il 1 Gennaio 2021 e il 30 Giugno 2021 (da fare) NO
#Step 2 Splitto l'RDD del dataset n°2 per il separatore di campo virgola

#split lines with ' , ' 
#apply filter to years equal to 2021
#filtro prendendo solo le righe per cui risultano non nulle le vaccinazioni a donne
#filtro per ottenere solo i valori nel 2021
data = data.map(lambda line : line.split(',')) \
           .filter(lambda x: x[0].split('-')[0] == '2021') \
           .map(lambda x: ( ( ( datetime.strptime( x[0][:10], '%Y-%m-%d' ).month ), #mese a numero
                           str(x[3]),#fascia d'età
                           str(x[11])), #regione
                           int(x[5]) #donne vaccinate
                           )        
           ) 

# facendo così non elimini la fascia d'età delle donne dove solo una donna si è vaccinata in quel mese ma mantieni le singole donne quando vai ad aggregare.
#quindi invece del .filter(lambda x: x[5] != '0') ho fatto 

# creo un indice 1 se i vaccini sono > 1 e 0 altrimenti, elimino se 0

data = data.map( lambda x: ( x[0], ( x[1], 1 ) if x[1] > 1 else ( x[1], 0 ) ) ).filter( lambda x: x[1][1] != 0 ).map( lambda x: ( ( x[0] ) , x[1][0] ) )
                                           
# si può anche mettere tutto insieme ma per ora è meglio capire quello che faccia

In [ ]:
data = data.groupByKey().mapValues(lambda x: (min(x), max(x), round(sum(x)/len(x)))) \
                  .sortBy(lambda x: x[0][1]) \
                  .sortBy(lambda x: x[0][0]) \
                  .map(lambda x: ( datetime.strptime(str(x[0][0]), "%m").strftime("%B"), x[0][1], x[0][2], x[1][0], x[1][1], x[1][2] )) \
                  .sortBy(lambda x: x[2]) 
                  

In [ ]:
data.take(25)

[('January', '12-19', 'Abruzzo', 2, 2, 2),
 ('January', '20-29', 'Abruzzo', 3, 120, 57),
 ('January', '30-39', 'Abruzzo', 2, 233, 101),
 ('January', '40-49', 'Abruzzo', 2, 295, 133),
 ('January', '50-59', 'Abruzzo', 3, 346, 163),
 ('January', '60-69', 'Abruzzo', 2, 217, 94),
 ('January', '70-79', 'Abruzzo', 2, 26, 8),
 ('January', '80-89', 'Abruzzo', 4, 34, 17),
 ('January', '90+', 'Abruzzo', 3, 44, 15),
 ('February', '12-19', 'Abruzzo', 2, 8, 4),
 ('February', '20-29', 'Abruzzo', 2, 146, 53),
 ('February', '30-39', 'Abruzzo', 2, 266, 81),
 ('February', '40-49', 'Abruzzo', 2, 613, 98),
 ('February', '50-59', 'Abruzzo', 2, 790, 124),
 ('February', '60-69', 'Abruzzo', 2, 320, 72),
 ('February', '70-79', 'Abruzzo', 2, 31, 15),
 ('February', '80-89', 'Abruzzo', 2, 866, 202),
 ('February', '90+', 'Abruzzo', 2, 328, 83),
 ('March', '12-19', 'Abruzzo', 2, 13, 5),
 ('March', '20-29', 'Abruzzo', 2, 229, 67),
 ('March', '30-39', 'Abruzzo', 2, 263, 82),
 ('March', '40-49', 'Abruzzo', 3, 520, 119)

In [ ]:
data.coalesce(1).saveAsTextFile("/content/drive/MyDrive/Colab Notebooks/output_query_2_2")

In [ ]:
sc.stop()